In [1]:
import pandas as pd
import numpy
import torch 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append("..")
from utils import pad_collate
from dataloader_comma import CommaDataset
from dataloader_nuscenes import NUScenesDataset
from model import VTN
import matplotlib.pyplot as plt 
from PIL import Image
import glob
import os
from utils import * 
import re

In [2]:
def mse_loss(input, target, mask, reduction="mean"):
        out = (input[~mask]-target[~mask])**2
        return out.mean() if reduction == "mean" else out 

In [10]:

p = '/data1/jessica/data/toyota/ckpts_final/'

experiments = os.listdir(p)
res = {}
res_ckpt = {}
for elem in experiments:
    path = p + elem + "/lightning_logs/" 
    versions =[ elem[-1 ]for elem in os.listdir(path)]
    versions = sorted(versions)
    version = f"version_{versions[-1]}"
    print(version, os.listdir(path))
    checkpoint_path = path + version + "/checkpoints/"
    if "checkpoints" not in os.listdir(path + version): continue
    files = os.listdir(checkpoint_path)

    task = []
    ckpt = []
    files = sorted(files)
    for filename in files: 
        if filename.endswith(".csv"):
            df = pd.read_csv(checkpoint_path + filename)
            df.columns = ['preds', 'targets']
            m = (df['targets'] > 50).astype(bool) #| 
            m = (df['targets'] == 0).astype(bool) 
            
            loss3 = torch.sqrt(mse_loss(torch.tensor(df['preds']),  torch.tensor(df['targets']), torch.tensor(m)))
            task.append(loss3.item())
        if filename.endswith(".ckpt"):
            ckpt.append(checkpoint_path + '/' + filename)
    res[elem] = task
    res_ckpt[elem] = ckpt


version_0 ['version_0']
version_0 ['version_0']
version_6 ['version_4', 'version_6', 'version_5']
version_3 ['version_3', 'version_0', 'version_2', 'version_1']
version_4 ['version_4', 'version_3']
version_0 ['version_0']
version_0 ['version_0']
version_7 ['version_4', 'version_3', 'version_6', 'version_7', 'version_2', 'version_1', 'version_5']
version_2 ['version_0', 'version_2', 'version_1']
version_5 ['version_4', 'version_3', 'version_0', 'version_2', 'version_5']
version_5 ['version_4', 'version_3', 'version_2', 'version_5']
version_6 ['version_4', 'version_3', 'version_6', 'version_2', 'version_1', 'version_5']
version_0 ['version_0']
version_5 ['version_4', 'version_5']
version_2 ['version_2', 'version_1']
version_1 ['version_0', 'version_1']
version_0 ['version_0']
version_1 ['version_0', 'version_1']
version_7 ['version_4', 'version_3', 'version_6', 'version_0', 'version_7', 'version_1', 'version_5']
version_0 ['version_0']
version_8 ['version_7', 'version_8']
version_2 ['ver

In [11]:
res

{'ckpts_final_comma_distance_resnet': [11.072215789000163],
 'ckpts_final_comma_multitask_none': [4.755451477304886, 6.336068743116705],
 'ckpts_final_nuscenes_angle_vit': [24.18435515055251],
 'ckpts_final_nuscenes_multitask_none': [17.889306924574445,
  13.996815372694005],
 'ckpts_final_nuscenes_multitask_resnet': [31.21839503709648,
  34.463115414811156],
 'ckpts_final_comma_multitask_resnet': [2.931496294513174, 9.38132235089526],
 'ckpts_final_comma_angle_resnet': [3.3228762031705306],
 'ckpts_final_nuscenes_multitask_vit': [23.517274360016717,
  29.393134528473368],
 'ckpts_final_nuscenes_distance_vit': [],
 'ckpts_final_nuscenes_multitask_clip': [21.018238515699572,
  16.40901270288182],
 'ckpts_final_nuscenes_angle_none': [18.496407596431748],
 'ckpts_final_comma_distance_none': [8.493491488816435],
 'ckpts_final_nuscenes_angle_clip': [17.000516100779752],
 'ckpts_final_nuscenes_distance_clip': [15.15478176181139],
 'ckpts_final_comma_multitask_clip': [],
 'ckpts_final_nuscene

In [12]:
rows = []
for elem in res.keys():
    splitted = elem.split("_")
    data = splitted[2]
    task = splitted[3]
    backbone = splitted[4]
    rows.append({"Task": task, "Backbone": backbone, "Dataset": data, "t-RMSE": res[elem][0] if len(res[elem]) == 1 else res[elem]})



In [13]:
rows

[{'Task': 'distance',
  'Backbone': 'resnet',
  'Dataset': 'comma',
  't-RMSE': 11.072215789000163},
 {'Task': 'multitask',
  'Backbone': 'none',
  'Dataset': 'comma',
  't-RMSE': [4.755451477304886, 6.336068743116705]},
 {'Task': 'angle',
  'Backbone': 'vit',
  'Dataset': 'nuscenes',
  't-RMSE': 24.18435515055251},
 {'Task': 'multitask',
  'Backbone': 'none',
  'Dataset': 'nuscenes',
  't-RMSE': [17.889306924574445, 13.996815372694005]},
 {'Task': 'multitask',
  'Backbone': 'resnet',
  'Dataset': 'nuscenes',
  't-RMSE': [31.21839503709648, 34.463115414811156]},
 {'Task': 'multitask',
  'Backbone': 'resnet',
  'Dataset': 'comma',
  't-RMSE': [2.931496294513174, 9.38132235089526]},
 {'Task': 'angle',
  'Backbone': 'resnet',
  'Dataset': 'comma',
  't-RMSE': 3.3228762031705306},
 {'Task': 'multitask',
  'Backbone': 'vit',
  'Dataset': 'nuscenes',
  't-RMSE': [23.517274360016717, 29.393134528473368]},
 {'Task': 'distance', 'Backbone': 'vit', 'Dataset': 'nuscenes', 't-RMSE': []},
 {'Task':

In [14]:
df = pd.DataFrame(rows)
df

,Task,Backbone,Dataset,t-RMSE
0,distance,resnet,comma,11.072216
1,multitask,none,comma,"[4.755451477304886, 6.336068743116705]"
2,angle,vit,nuscenes,24.184355
3,multitask,none,nuscenes,"[17.889306924574445, 13.996815372694005]"
4,multitask,resnet,nuscenes,"[31.21839503709648, 34.463115414811156]"
5,multitask,resnet,comma,"[2.931496294513174, 9.38132235089526]"
6,angle,resnet,comma,3.322876
7,multitask,vit,nuscenes,"[23.517274360016717, 29.393134528473368]"
8,distance,vit,nuscenes,[]
9,multitask,clip,nuscenes,"[21.018238515699572, 16.40901270288182]"


In [15]:
import pandas as pd

# The list of dictionaries
data = rows

# Convert the list to a DataFrame
df = pd.DataFrame(data)

# Pivot the DataFrame
df_pivot = pd.pivot_table(df, values='t-RMSE', index=['Dataset', 'Backbone'], columns='Task', aggfunc=lambda x: x)

# Rename the columns
df_pivot.columns = ['t-RMSE ' + col for col in df_pivot.columns]

# Reset the index
df_pivot = df_pivot.reset_index()

# Display the pivoted DataFrame
print(df_pivot.to_latex())


\begin{tabular}{llllll}
\toprule
{} &   Dataset & Backbone & t-RMSE angle & t-RMSE distance &                          t-RMSE multitask \\
\midrule
0 &     comma &     clip &     2.141397 &             NaN &                                        [] \\
1 &     comma &     none &     5.325767 &        8.493491 &    [4.755451477304886, 6.336068743116705] \\
2 &     comma &   resnet &     3.322876 &       11.072216 &     [2.931496294513174, 9.38132235089526] \\
3 &     comma &      vit &          NaN &       12.687749 &                                       NaN \\
4 &  nuscenes &     clip &    17.000516 &       15.154782 &   [21.018238515699572, 16.40901270288182] \\
5 &  nuscenes &     none &    18.496408 &       24.478184 &  [17.889306924574445, 13.996815372694005] \\
6 &  nuscenes &   resnet &          NaN &              [] &   [31.21839503709648, 34.463115414811156] \\
7 &  nuscenes &      vit &    24.184355 &              [] &  [23.517274360016717, 29.393134528473368] \\
\bottomrule


/tmp/ipykernel_2057466/4291682410.py:19: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df_pivot.to_latex())


In [16]:
df_pivot

,Dataset,Backbone,t-RMSE angle,t-RMSE distance,t-RMSE multitask
0,comma,clip,2.141397,NaN,[]
1,comma,none,5.325767,8.493491,"[4.755451477304886, 6.336068743116705]"
2,comma,resnet,3.322876,11.072216,"[2.931496294513174, 9.38132235089526]"
3,comma,vit,NaN,12.687749,NaN
4,nuscenes,clip,17.000516,15.154782,"[21.018238515699572, 16.40901270288182]"
5,nuscenes,none,18.496408,24.478184,"[17.889306924574445, 13.996815372694005]"
6,nuscenes,resnet,NaN,[],"[31.21839503709648, 34.463115414811156]"
7,nuscenes,vit,24.184355,[],"[23.517274360016717, 29.393134528473368]"
